In [ ]:
import os
import tensorflow as tf

import shutil

import read_data as rd
import matplotlib.pyplot as plt

tf.reset_default_graph()
      

In [ ]:
""" First downlaod the images and associated labels"""
dataset_path = 'data'
if os.path.exists(dataset_path):
    found = 1
else:
    os.mkdir(os.path.join(os.getcwd(),dataset_path))
    
model_dir = '/kernel_log'
if os.path.exists(model_dir):
    found = 1
else:
    os.mkdir(os.path.join(os.getcwd(),model_dir))

rd.download_extract(dataset_path)
dataset = rd.Dataset(dataset_path)

In [ ]:
px= tf.placeholder(tf.float32, [None,28,28,3]) 
py= tf.placeholder(tf.float32, [None,2]) 
ptr = tf.placeholder(tf.bool,shape=None)
from model import gender_classifier,wce_loss
pr = gender_classifier(px,ptr)
cost = wce_loss(py,pr)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
prediction = tf.argmax(pr, 1)
correct_answer = tf.argmax(py, 1)
equality = tf.equal(correct_answer,prediction)
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

In [ ]:
batch_size = 32
num_train_batches = int(np.floor(dataset.X_train.shape[0]/batch_size))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epochn in range(1):
        for batchn in range(num_train_batches):
            in1 = batchn*batch_size
            in2 = in1 + batch_size
            xb_train,yb_train = get_batch(dataset.X_train,dataset.Y_train,in1,in2,batch_size)
            pred = sess.run(pr,feed_dict = {px : xb_train,ptr:True})
            batch_loss = sess.run(cost,feed_dict={py:yb_train,pr:pred})
            if (batchn%100)==0:
                nval = dataset.X_validate.shape[0]
                nbval = np.floor(nval/500)
                in1 = int((nbval-2)*500)
                in2 = in1 + 500
                xb_val,yb_val = get_batch(dataset.X_validate,dataset.Y_validate,in1,in2,500)
                pred = sess.run(pr,feed_dict = {px : xb_val,ptr:False})
                acc = sess.run(accuracy,feed_dict={py:yb_val,pr:pred})
                print('loss:'+str(batch_loss)+', accuracy: '+str(acc))
        
        
    

In [ ]:
data_batch, indices = dataset.get_image_batch(16)
gender_class = sess.run(prediction,feed_dict={px:data_batch,ptr:False})
